In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

# Classic Metropolis-Hastings

We start with the classic Metropolis-Hasting algorithm. Let $p(x)$ be the target probability distribution and $q(x)$ represent the proposal distribution. The algorithm can be summarised as follows.

1. start walk from an initial position $x_0$
2. propose a new psotion $x_1$ from $q(x)$ conditional on $x_0$, i.e., $x_1 \sim q(x_1|x_0)$
3. accept the new position with the proability $\alpha(x_0, x_1) = min\left[1, \frac{p(x_1)q(x_0|x_1)}{p(x_0)q(x_1|x_0)}\right]$
4. if accepted move the initial position to $x_1$ and return to step 1
